<a href="https://colab.research.google.com/github/proyectosRVyderivados/cristina/blob/main/ARBITRAJE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Ejemplo sencillo
Puede ver la Hoja1 del documento de Excel.

Supongamos dos fruterías donde venden el mismo tipo de manzana.

Tienda A: precio 100 céntimos/Kg
Tienda B: precio 160 céntimos/Kg
Operamos bajo el supuesto de que en cada tienda podemos comprar o vender las manzanas.

Estrategia de arbitraje:

Compramos las manzanas en la tiena A, donde el precio es más barato
Vendemos las manzanas en la tienda B, donde el precio es más caro
La estrategia consiste en comprar el activo infravalorado y vender el activo sobrevalorado.

De esta forma estaríamos ganando la diferencia.

Introduciendo intereses
Supongamos que el caso anterior de compraventa no se produce simultáneamente en el tiempo y que necesitamos financiar nuestra compra durante un tiempo.

Duración de la operación: 1 año
Tipo de interés: 10% anual
En t=0 pedimos un prestamo de 100 para comprar en la tienda A
En t=0 compramos las manzanas en la tienda A por 100
En t=1 vendemos las manzanas en la tienda B por 160
En t=1 devolvemos el préstamo 100*(1+0.1) = 110
Beneficio obtenido:
Método 1: 160-100-10 = 50 en t=1
Método 2: 100-100+160-110 = 50 en t=1

In [2]:
pip install numpy-financial

In [3]:
pip install numpy-financial
import numpy_financial as npf
import numpy as np             # la librería numpy nos permitirá trabajar con ndarrays

# DATOS
bonoA = [0, 10, 1010]
bonoB = [0, 200, 1200]
etti = [.05, .2]

# PRECIO DE UN BONO CON LA ETTI
def precio_etti(flujos, etti):
    precio = 0
    for i in range(1, len(bonoA)):
        precio += flujos[i] * (1+etti[i-1])**-i
    return precio

# Precios de los Bonos A y B
precioA = precio_etti(bonoA, etti)
precioB = precio_etti(bonoB, etti)
print(f"El precio del bono A es {precioA}")
print(f"El precio del bono B es {precioB}")
# Flujos de caja de los bonos A y B
bonoA[0] = -precioA
bonoB[0] = -precioB

# Calcular la TIR de los bonos
tirA = npf.irr(bonoA)
tirB = npf.irr(bonoB)
print(f"La TIR del bono A es: {tirA:.4%}")
print(f"La TIR del bono B es: {tirB:.4%}")

# Calcular la duración de Macaulay
def duracion(bono):
    tir =npf.irr(bono)
    precio = 0
    numerador = 0
    for i in range(1, len(bonoA)):
        precio += bono[i] * (1+tir)**-i
        numerador += i * bono[i] * (1+tir)**-i
    return numerador / precio

print(f"La duration del bono A es {duracion(bonoA)}")
print(f"La duration del bono B es {duracion(bonoB)}")

# Nuevo precio del bono A en el mercado desequilibrado
precioAprima = npf.npv(tirB, bonoA) + precioA # sumamos el antiguo precio de A ya que va en la posición [0] del array con signo negativo
print(f"El nuevo precio del bono A en el mercado desequilibrado es {precioAprima}")
bonoAprima =[-precioAprima, bonoA[1], bonoA[2]]

# Bono C = B - 20A'
# bonoC = bonoB - 20 * bonoAprima  # esto da error, necesitamos trabajar con ndarrays
bonoB_arr = np.array(bonoB)
bonoAprima_arr =np.array(bonoAprima)
bonoC_arr = bonoB_arr - 20 * bonoAprima_arr
print(f"El bono C es {bonoC_arr}")

# Ajustamos con un préstamo
prestamo = np.array([bonoC_arr[2] / (1+etti[1])**2, 0, -bonoC_arr[2]])
print(f"El préstamo es {prestamo}")

# Cartera Total
total = bonoC_arr + prestamo
print(f"Cartera resultante del arbitraje {total}")

SyntaxError: ignored